# Ollama: Custom Rest client

In [1]:
%%capture output
# %load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

BASE_PATH         = '../..'
API_PATH          = f'{BASE_PATH}/chat-bot-api'
LIB_PATH          = f'{BASE_PATH}/lib'

os.environ['TMP_PATH']         = f'{BASE_PATH}/tmp'
os.environ['DATASET_PATH']     = f'{BASE_PATH}/datasets'
os.environ['WEIGHTS_PATH']     = f'{BASE_PATH}/weights'
os.environ['METRICS_PATH']     = f'{BASE_PATH}/metrics'
os.environ['MONGODB_URL']      = 'mongodb://0.0.0.0:27017'
os.environ['MONGODB_DATABASE'] = 'chatbot'
os.environ['CHROMA_HOST']      = '0.0.0.0'
os.environ['CHROMA_PORT']      = '9090'

sys.path.append(LIB_PATH)
sys.path.append(API_PATH)

In [3]:
from rest.ollama import OllamaApiClient
from services import OllamaSentenceEmbeddingService
from recommender.chatbot.movie import MovieRecommendationsOutputParser, MovieRecommenderChatBotFactory

<Figure size 640x480 with 0 Axes>

## Setup

In [4]:
user_profile = """
User profile:
- Name: Adrian
- Genre: Male
- Age: 32
- Nationality: Argentina
- Studies: Engineering
- Work: Software Engineer
- Movie preferences:
  - Released from 1970
  - Genres: thiller, suspense, science fiction, love, comedy
"""

candidates = """
Candidate movies (with rating):
- Iron Man 3: 3.3
- Star Trek Into Darkness: 3.5
- Man of Steel: 3.1
- Despicable Me 2: 3.4
- Wolverine, The: 2.8
- Kick-Ass 2: 3.0
- Thor: The Dark World: 3.1
- The Lego Movie: 3.6
- Captain America: The Winter Soldier: 3.6
- The Amazing Spider-Man 2: 2.9
- X-Men: Days of Future Past: 3.6
- Mission: Impossible - Rogue Nation: 3.4
- Guardians of the Galaxy: 3.7
- Crippled Avengers: 2.5
- Penguins of Madagascar: 3.4
- Star Wars: Episode V - The Empire Strikes Back: 4.2
- Star Wars: Episode VII - The Force Awakens: 3.2
- Avengers: Age of Ultron: 3.3
- Ant-Man: 3.4
- Fantastic Four: 1.8
- Guardians of the Galaxy 2: 3.4
- Captain America: Civil War: 3.3
- Doctor Strange: 3.5
- X-Men: Apocalypse: 2.9
- Marvel One-Shot: Item 47: 3.0
- Furious 7: 3.1
- Ghost in the Shell: Stand Alone Complex - The Laughing Man: 4.0
- Absolutely Anything: 3.3
- Next Avengers: Heroes of Tomorrow: 5.0
- Iron Man & Captain America: Heroes United: 3.0
- Batman v Superman: Dawn of Justice: 2.6
- Star Trek: The Motion Picture: 3.1
- Iron Man & Hulk: Heroes United: 3.0
- Wonder Woman: 2.0
- Captain Fantastic: 3.8
- Now You See Me 2: 3.5
- LEGO DC Comics Super Heroes: Batman: Be-Leaguered: 2.0
- While You Were Fighting: A Thor Mockumentary: 3.8
- Marvel One-Shot: All Hail the King: 3.0
- Marvel One-Shot: The Consultant: 2.3
- Rogue One: A Star Wars Story: 3.5
- Superheroes: A Never-Ending Battle: 4.0
- xXx: Return of Xander Cage: 3.4
- The Lego Batman Movie: 3.9
- Marvel One-Shot: Agent Carter: 4.0
- Marvel One-Shot: A Funny Thing Happened on the Way to Thor's Hammer: 4.0
- Logan: 3.8
- Kong: Skull Island: 3.1
- Despicable Me 3: 2.7
- Paragraph 78: Film One: 3.0
- X-Files: Fight the Future, The: 3.4
- Avengers, The: 1.8
- Comic Book Confidential: 4.0
- Captain America: 2.0
- Austin Powers: The Spy Who Shagged Me: 2.8
- Mystery Men: 2.9
- Fantastic Four: 2.5
- Sky High: 2.9
- Starman: 3.7
- X-Men: 3.5
- Supergirl: 2.2
- Demolition Man: 3.3
- X-Men: The Last Stand: 3.0
- Punisher, The: 2.4
- Superman Returns: 2.7
- Blankman: 2.0
- Spider-Man 3: 2.7
- Fantastic Four: Rise of the Silver Surfer: 2.2
- Spider-Man: 3.4
- Men in Black II: 2.7
- Superhero Movie: 2.8
- Iron Man: 3.8
- Incredible Hulk, The: 3.1
- Hancock: 3.2
- Watchmen: 3.7
- Cowboy Bebop: The Movie: 3.9
- X2: X-Men United: 3.6
- Punisher: War Zone: 2.8
- Comic Book Villains: 1.0
- Mystery Science Theater 3000: The Movie: 4.1
- X-Men Origins: Wolverine: 3.0
- Star Trek: 3.8
- Fanboys: 2.8
- Avatar: 3.7
- Phantom, The: 3.1
- Iron Man 2: 3.4
- Despicable Me: 3.6
- Green Hornet, The: 2.7
- Thor: 3.1
- Spider-Man 2: 3.5
- X-Men: First Class: 3.5
- Captain America: The First Avenger: 3.2
- Incredibles, The: 3.8
- Avengers, The: 3.6
- Superheroes: 4.0
- Mission: Impossible - Ghost Protocol: 3.6
- Comic-Con Episode IV: A Fan's Hope: 2.0
- Men in Black III: 3.2
- Amazing Spider-Man, The: 3.1
- Rise of the Guardians: 3.7
"""

In [5]:
OllamaApiClient().models()


[
    'gemma:7b',
    'gemma-7b:latest',
    'llama2:7b-chat',
    'llama2-7b-chat:latest',
    'llama3:instruct',
    'llama3:text',
    'llama3-8b-instruct:latest',
    'llama3-rec:latest',
    'mistral:instruct',
    'mistral:latest',
    'mistral-instruct:latest',
    'mixtral:latest',
    'mxbai-embed-large:latest',
    'snowflake-arctic-embed:latest'
]

In [6]:
%%time

PROMPT = """
{user_profile}

{user_history}

{candidates}

Recommend {limit} movies from candidate movies list, based on user profile and seen movies.
Return only one list of recommendations with next format:

Number. Title (Release year): Synopsis.
"""

chatbot_client = MovieRecommenderChatBotFactory.create(
    prompt = PROMPT,
    model  = 'llama3-rec:latest'
)

result = chatbot_client(
    user_profile = user_profile, 
    candidates   = candidates,
    limit        = 10
)

result.metadata['recommendations']

CPU times: user 3.84 ms, sys: 519 µs, total: 4.36 ms
Wall time: 10.2 s



[
    {
        'number': 1,
        'title': 'Star wars: episode v - the empire strikes back',
        'description': 'A classic sci-fi film that explores the complexities of friendship and loyalty as luke skywalker confronts darth vader.',
        'release': '1980'
    },
    {
        'number': 2,
        'title': 'Cowboy bebop: the movie',
        'description': 'An animated sci-fi film that combines elements of jazz, action, and drama, set in a futuristic world where humans and aliens coexist.',
        'release': '2001'
    },
    {
        'number': 3,
        'title': 'Watchmen',
        'description': 'A dark and gritty superhero deconstruction that explores the complexities of heroism and morality in a world on the brink of nuclear war.',
        'release': '2009'
    },
    {
        'number': 4,
        'title': 'Incredibles, the',
        'description': 'An animated superhero film that balances action, humor, and heart, as a family with superpowers tries to live a normal 

In [7]:
%%time

embedding0 = OllamaApiClient().embedding(
    model='llama3-rec:latest',
    prompt='When the avengers are divided over government oversight, captain america must choose between his loyalty and his moral'
)

len(embedding0)

CPU times: user 1.56 ms, sys: 947 µs, total: 2.51 ms
Wall time: 62.7 ms


4096

In [8]:
%%time

embedding1 = OllamaApiClient().embedding(
    model='llama3-rec:latest',
    prompt='The galaxy is on the brink of war as luke skywalker trains to face darth vader in a lightsaber duel.'
)

len(embedding1)

CPU times: user 1.68 ms, sys: 1.03 ms, total: 2.71 ms
Wall time: 61.2 ms


4096

In [9]:
%%time

embedding2 = OllamaApiClient().embedding(
    model='llama3-rec:latest',
    prompt='In this futuristic anime film, spike spiegel and his crew must stop a powerful crime lord from taking over the solar system.'
)

len(embedding2)

CPU times: user 2.67 ms, sys: 0 ns, total: 2.67 ms
Wall time: 61.2 ms


4096

In [10]:
%%time

embedding3 = OllamaApiClient().embedding(
    model='llama3-rec:latest',
    prompt='Set in an alternate universe, this superhero thriller follows a group of vigilantes as they try to prevent a catastrophic event.'
)

len(embedding3)

CPU times: user 2.74 ms, sys: 0 ns, total: 2.74 ms
Wall time: 62.9 ms


4096

In [11]:
%%time

embedding4 = OllamaApiClient().embedding(
    model='llama3-rec:latest',
    prompt='An ordinary lego minifigure becomes the "chosen one" to stop an evil tyrant from gluing the universe together.'
)

len(embedding4)

CPU times: user 2.96 ms, sys: 0 ns, total: 2.96 ms
Wall time: 62.1 ms


4096

In [12]:
%%time

embeddings = OllamaSentenceEmbeddingService(model='llama3-rec:latest').generate(
    texts = [
        'When the avengers are divided over government oversight, captain america must choose between his loyalty and his moral',
        'The galaxy is on the brink of war as luke skywalker trains to face darth vader in a lightsaber duel.',
        'In this futuristic anime film, spike spiegel and his crew must stop a powerful crime lord from taking over the solar system.',
        'Set in an alternate universe, this superhero thriller follows a group of vigilantes as they try to prevent a catastrophic event.',
        'An ordinary lego minifigure becomes the "chosen one" to stop an evil tyrant from gluing the universe together.'
    ]
)

embeddings.shape

CPU times: user 8.93 ms, sys: 1.81 ms, total: 10.7 ms
Wall time: 305 ms


(5, 4096)

In [13]:
embeddings[0] == embedding0, embeddings[1] == embedding1, embeddings[2] == embedding2, embeddings[3] == embedding3, embeddings[4] == embedding4


(
    array([ True,  True,  True, ...,  True,  True,  True]),
    array([ True,  True,  True, ...,  True,  True,  True]),
    array([ True,  True,  True, ...,  True,  True,  True]),
    array([ True,  True,  True, ...,  True,  True,  True]),
    array([ True,  True,  True, ...,  True,  True,  True])
)

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [15]:
embeddings[0].reshape(-1).shape

(4096,)

In [16]:
sim = lambda a, b: cosine_similarity([a], [b])[0][0]
dist = lambda a, b: round(1 - sim(a, b), 10)

In [17]:
dist(embedding0, embeddings[0]), dist(embedding1, embeddings[1]), dist(embedding2, embeddings[2])

(0.0, 0.0, 0.0)

In [18]:
dist(embedding1, embeddings[0])

0.7174359533

In [19]:
import ollama

ollama.embeddings(model='mxbai-embed-large', prompt='text')['embedding']


[
    -0.11157584190368652,
    -0.8045521974563599,
    -0.03237351030111313,
    -0.18947777152061462,
    0.6801958084106445,
    -0.3505572974681854,
    -0.05846791714429855,
    0.0638943612575531,
    0.3412528932094574,
    0.3414247930049896,
    1.1269447803497314,
    -0.218951016664505,
    1.0048518180847168,
    -0.377706378698349,
    -1.2056206464767456,
    0.14942258596420288,
    -0.13409321010112762,
    -0.960737943649292,
    -0.0323624387383461,
    -0.4753531813621521,
    -0.044406890869140625,
    -0.4753595292568207,
    -1.1712316274642944,
    -0.1430373191833496,
    -0.2997286319732666,
    0.8523834943771362,
    -0.2515360116958618,
    0.013482730835676193,
    0.6984204053878784,
    0.07255331426858902,
    -0.8346003890037537,
    -0.8376440405845642,
    -0.18727707862854004,
    -0.45687156915664673,
    -0.1811101883649826,
    0.43814530968666077,
    0.6860730648040771,
    -0.9552104473114014,
    -0.7669818997383118,
    -1.282824993133545,
